# Preparação de Dados e Análise Exploratória

## 1. Importando bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from category_encoders import OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn import metrics
import holidays
from sklearn.preprocessing import LabelEncoder

## 2. Importando dados

In [ ]:
mes_02 = pd.read_csv('../data/month_2.csv')
mes_03 = pd.read_csv('../data/month_3.csv')
mes_04 = pd.read_csv('../data/month_4.csv')
mes_05 = pd.read_csv('../data/month_5.csv')
mes_06 = pd.read_csv('../data/month_6.csv')
cadastro = pd.read_csv('../data/informacao_cadastral.csv')

meses = pd.concat([mes_02, mes_03, mes_04, mes_05, mes_06])

df = pd.merge(meses, cadastro, on=['clientCode', 'clientIndex',], how='left')

## 3. Tratamento de dados

Essa etapa é muito importante para garantir que os dados estejam prontos para serem utilizados no modelo de machine learning. Aqui, vamos tratar valores ausentes, converter tipos de dados, criar novas variáveis e fazer outras transformações necessárias.

### Removendo colunas desnecessárias

In [ ]:
df = df.drop(columns=['initialIndex', 'meterSN', 'cidade', 'contratacao', 'condIndex' ])
df.drop(columns=['gatewayGeoLocation.alt','gatewayGeoLocation.lat', 'gatewayGeoLocation.long'], inplace=True)

### Preenchendo valores faltantes

In [ ]:
df['rssi'] = df['rssi'].fillna(0)

for column in ['bairro', 'categoria', 'perfil_consumo', 'condCode']:
    df[column] = df[column].fillna(df[column].mode()[0])


## 4. Dados Meteorológicos

Adiciona features de dados meteorológicos ao dataset. Trazendo informações de temperatura, umidade, pressão atmosférica e direção do vento.

### Importando dados meteorológicos

In [ ]:
df_meteorologicos = pd.read_csv('../data/generatedBy_react-csv.csv', sep=";")

### Removendo colunas desnecessárias

In [ ]:
df_meteorologicos = df_meteorologicos.drop(columns=['Radiacao (KJ/m²)','Hora (UTC)'])

### Alterando o nome das colunas

In [ ]:
df_meteorologicos = df_meteorologicos.rename(columns={'Dir. Vento (m/s)': 'Dir. Vento (°)'})

### Verificando valores faltantes

In [ ]:
df_meteorologicos.isnull().sum()

### Função para preencher valores faltantes

In [ ]:
def tratar_valores_nulos(df, col_min, col_instant, col_max):

    df[col_min].fillna(df[col_instant], inplace=True)
    df[col_min].fillna(df[col_max], inplace=True)

    df[col_instant].fillna(df[col_min], inplace=True)
    df[col_instant].fillna(df[col_max], inplace=True)

    df[col_max].fillna(df[col_instant], inplace=True)
    df[col_max].fillna(df[col_min], inplace=True)

    return df

### Preenchendo valores faltantes

In [ ]:
tratar_valores_nulos(df_meteorologicos,'Temp. Min. (C)', 'Temp. Ins. (C)', 'Temp. Max. (C)')
tratar_valores_nulos(df_meteorologicos,'Umi. Min. (%)', 'Umi. Ins. (%)', 'Umi. Max. (%)')
tratar_valores_nulos(df_meteorologicos,'Pto Orvalho Min. (C)', 'Pto Orvalho Ins. (C)', 'Pto Orvalho Max. (C)')
tratar_valores_nulos(df_meteorologicos,'Pressao Min. (hPa)', 'Pressao Ins. (hPa)', 'Pressao Max. (hPa)')

### Alterando o padrão de valores de "," para "."

In [ ]:
for column in ['Temp. Ins. (C)', 'Temp. Max. (C)', 'Temp. Min. (C)',
       'Umi. Ins. (%)', 'Umi. Max. (%)', 'Umi. Min. (%)',
       'Pto Orvalho Ins. (C)', 'Pto Orvalho Max. (C)', 'Pto Orvalho Min. (C)',
       'Pressao Ins. (hPa)', 'Pressao Max. (hPa)', 'Pressao Min. (hPa)',
       'Vel. Vento (m/s)', 'Dir. Vento (°)', 'Raj. Vento (m/s)',
       'Chuva (mm)']:
    df_meteorologicos[column] = pd.to_numeric(df_meteorologicos[column].str.replace(',', '.'), errors='coerce')

### Verifica se os valores foram alterados

In [ ]:
for column in ['Vel. Vento (m/s)', 'Dir. Vento (°)', 'Raj. Vento (m/s)','Chuva (mm)']:
        df_meteorologicos[column].fillna(df_meteorologicos[column].mean(), inplace=True)

### Definindo a data como índice

In [ ]:
df_meteorologicos['Data'] = pd.to_datetime(df_meteorologicos['Data'], dayfirst=True)

### Agrupando os dados por dia

In [ ]:
df_meteorologicos = round(df_meteorologicos.groupby(df_meteorologicos['Data'].dt.date, as_index=False).mean(), 2)

### Alterando o nome de "Data" para "date"

In [ ]:
df_meteorologicos = df_meteorologicos.rename(columns={'Data': 'date'})

## 5. Códificação de variáveis categóricas

Essa etapa é necessária para transformar as variáveis categóricas em numéricas.

### Códificando variáveis categóricas

In [ ]:
LabelEncoder = LabelEncoder()

df['clientCode'] = LabelEncoder.fit_transform(df['clientCode'])
df['bairro'] = LabelEncoder.fit_transform(df['bairro'])
df['categoria'] = LabelEncoder.fit_transform(df['categoria'])
df['condCode'] = LabelEncoder.fit_transform(df['condCode'])

In [ ]:
input_type_mapping = {
    'leituraRemota': 0,
    'DI1': 1,
    'DI2': 2,
    'DI3': 2,
    'DI4': 4,
    'DI5': 5,
    'DI6': 6,
    'DI7': 7,
    'DI8': 8,
}

df['inputType'] = df['inputType'].map(input_type_mapping)

df

### Códificando a coluna "model" com One Hot Encoding

In [ ]:
codificadorModel = OneHotEncoder(cols=['model'])
df = codificadorModel.fit_transform(df)
df.rename(columns={'model_1': 'IG1K-L-v2', 'model_2': 'Infinity V2'}, inplace=True)
df

### Removendo caracteres especiais

In [ ]:
moda = df['perfil_consumo'].mode()[0]
df = df.replace('-', moda)

### Códificando a coluna "perfil_consumo" com One Hot Encoding

In [ ]:
codificadorPerfilConsumo = OneHotEncoder(cols=['perfil_consumo'])
df = codificadorPerfilConsumo.fit_transform(df)
df

In [ ]:
df_auxiliar = pd.merge(meses, cadastro, on=['clientCode', 'clientIndex',], how='left')

colunasPerfilConsumo = df[['perfil_consumo_1', 'perfil_consumo_2', 'perfil_consumo_3', 
                             'perfil_consumo_4', 'perfil_consumo_5', 'perfil_consumo_6']]

indices = []

for coluna in colunasPerfilConsumo:
    indice = df[df[coluna] == 1].index[0] if not df[df[coluna] == 1].empty else None
    indices.append(indice)

valorIndice = []

for indice in indices:
    valor = df_auxiliar.loc[indice,'perfil_consumo']
    valorIndice.append(valor)

df = df.rename(columns={'perfil_consumo_1': valorIndice[0], 'perfil_consumo_2': valorIndice[1], 'perfil_consumo_3': valorIndice[2], 'perfil_consumo_4': valorIndice[3], 'perfil_consumo_5': valorIndice[4], 'perfil_consumo_6': valorIndice[5] })

df

## 6. Feature "Dia da Semana"

Essa feature é importante para identificar se o dia da semana influencia no consumo de energia. Como a variável "data" está no formato datetime, podemos extrair o dia da semana, sendo 0 = segunda-feira e 6 = domingo.

In [ ]:
# Converter a coluna 'data_completa' para o tipo datetime
df['datetime'] = pd.to_datetime(df['datetime'])

# Criar a coluna apenas com a hora
df['hora'] = df['datetime'].dt.strftime('%H').astype(int)

# Criar a coluna apenas com o mes
df['mes'] = df['datetime'].dt.strftime('%m').astype(int)

# Criar a coluna com o dia da semana
df['dia_da_semana'] = df['datetime'].dt.weekday

df

## 7. Feature "Consumo por dia"

Essa feature é importante para identificar o consumo de energia por dia.

### Preenchendo valores faltantes no "gain" com 1

In [ ]:
df['gain'] = df['gain'].fillna(1)
df['medidor'] = df['pulseCount'] * df['gain']

### Criando a feature "gasto", que é a multiplicação do "consumo" pelo "gain", agrupando por dia

In [ ]:
df['gasto'] = 0.0

df = df.sort_values(by=['clientCode', 'datetime'])

df['gasto'] = df.groupby(['clientCode', 'clientIndex'])['medidor'].diff()

df['gasto'] = df['gasto'].fillna(0)

resultado = df

df = df.sort_values(by=['datetime'])

print(df[['clientCode', 'medidor', 'gasto', 'datetime']])

print(df[['clientCode', 'medidor', 'gasto', 'datetime']])
print("\nLinhas com 'gasto' diferente de 0:")
print(resultado)

In [ ]:
df.sort_values(by=['clientCode', 'datetime'], inplace=True)
df.head(5)

In [ ]:
# Converter a coluna 'datetime' para o formato de data
df['datetime'] = pd.to_datetime(df['datetime'])

# Criar uma nova coluna 'date' contendo apenas a df (sem a hora)
df['date'] = df['datetime'].dt.date

# Agrupar por 'clientCode' e 'date' e somar o 'gasto' para obter o consumo diário
daily_consumption = df.groupby(['clientCode', 'clientIndex', 'date'])['gasto'].sum().reset_index()

# daily_consumption['dia_da_semana'] = pd.to_datetime(df['date']).dt.weekday
# daily_consumption['mes'] = pd.to_datetime(df['date']).dt.month
# daily_consumption['hora'] = pd.to_datetime(df['date']).dt.hour

# Colunas
colunasDf = ['clientCode', 'clientIndex', 'date', 'hora', 'dia_da_semana', 'mes', 'bairro', 'categoria', 'IG1K-L-v2', 'Infinity V2', 'inputType', 'medidor', 'feriado']

df = pd.merge(daily_consumption, 
                df[colunasDf].drop_duplicates(subset=['clientCode','clientIndex', 'date']),
                on=['clientCode','clientIndex', 'date'],
                how='left')

### 8. Features "Feriados"

Feature que identifica se o dia é feriado ou não. Ou seja, se o dia é um feriado, o valor é 1, caso contrário, o valor é 0.

In [ ]:
# Criar a lista de feriados para o Brasil
br_holidays = holidays.Brazil()

# Converter a coluna 'datetime' para o formato datetime do pandas
df['datetime'] = pd.to_datetime(df['datetime'])

# Criar uma nova coluna 'feriado', onde 1 indica que a data é feriado e 0 indica que não é
df['feriado'] = df['datetime'].dt.date.apply(lambda x: 1 if x in br_holidays else 0)


### 9. Feature "Condições Climáticas"

Junta as features de temperatura, umidade, pressão atmosférica e direção do vento ao dataset.

In [ ]:
df_atualizado = pd.merge(df, df_meteorologicos, on='date', how='left')
df_atualizado

### 10. Feature "Gasto por hora"

Essa feature é importante para identificar o consumo por hora.

In [ ]:
df_atualizado['datetime'] = pd.to_datetime(df_atualizado['date'].astype(str) + ' ' + df_atualizado['hora'].astype(str) + ':00')

# Filtrar onde 'gasto' não é zero
df_atualizado = df_atualizado[df_atualizado['gasto'] != 0]

# Ordenar os dados
df_atualizado = df_atualizado.sort_values(by=['clientCode', 'clientIndex', 'datetime'])

# Calcular a diferença de tempo em horas
df_atualizado['diferenca_tempo'] = df_atualizado.groupby(['clientCode', 'clientIndex'])['datetime'].diff().dt.total_seconds().fillna(3600) / 3600

# Calcular o gasto por hora
df_atualizado['gasto_por_hora'] = round(df_atualizado['gasto'] / df_atualizado['diferenca_tempo'], 10)

# Remover a coluna de diferença de tempo
df_atualizado = df_atualizado.drop(columns=['diferenca_tempo'])

In [ ]:
df_atualizado = df_atualizado.drop(columns=['datetime'])

### 11. Exportando o dataset para o usar no modelo de machine learning

In [ ]:
df_atualizado.to_csv('../data/df_atualizado.csv', index=False)